In [1]:
import os
import random
import shutil
import copy
import time
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
from glob import glob
from skimage.io import imread
from os import listdir
import cv2

from scipy import ndimage
from scipy.spatial import distance
from sklearn.cluster import KMeans
#import tensorflow as tf


C:\Users\alvaro.rlanceta\AppData\Local\anaconda3\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# Ruta de la carpeta principal
main_folder = "C:\\Users\\alvaro.rlanceta\\Documents\\tfm\\datasetstfm\\datasets_D"




In [3]:
import os
import cv2

def load_and_extract_features(folder, max_images_per_class=None):
    images = []
    labels = []
    descriptors_list = []
    sift = cv2.SIFT_create()
    count_per_class = {}

    for label in sorted(os.listdir(folder)):
        class_folder = os.path.join(folder, label)
        count_per_class[label] = 0  # Inicializar contador para cada clase

        for filename in os.listdir(class_folder):
            if max_images_per_class is not None and count_per_class[label] >= max_images_per_class:
                continue  # Pasar a la siguiente clase si ya se alcanzó el límite

            img_path = os.path.join(class_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                keypoints, descriptors = sift.detectAndCompute(img, None)
                if descriptors is not None:
                    images.append(img)
                    labels.append(int(label))
                    descriptors_list.append(descriptors)
                    count_per_class[label] += 1

            if max_images_per_class is not None and all(count >= max_images_per_class for count in count_per_class.values()):
                break  # Terminar completamente si todas las clases alcanzaron el límite

    return images, labels, descriptors_list


In [4]:
def build_histograms(kmeans, descriptors_list):
    histograms = []
    for descriptors in descriptors_list:
        histogram = np.zeros(len(kmeans.cluster_centers_))
        clusters = kmeans.predict(descriptors)
        for i in clusters:
            histogram[i] += 1
        histograms.append(histogram)
    return histograms



In [5]:
from sklearn.cluster import KMeans
import numpy as np

def kmeans_clustering(descriptors_list, K):
    # Concatenar todos los descriptores en una sola lista de numpy para k-means
    all_descriptors = np.vstack(descriptors_list)
    kmeans = KMeans(n_clusters=K, random_state=0).fit(all_descriptors)
    return kmeans


In [6]:
import datetime

# Tiempo inicial
start_time = datetime.datetime.now()

In [7]:
train_images, train_labels, train_descriptors = load_and_extract_features(main_folder + "\\train", 20000)

# Clustering para formar el vocabulario
kmeans = kmeans_clustering(train_descriptors, K=200)  # Número de visual words

# Construir histogramas
train_histograms = build_histograms(kmeans, train_descriptors)

In [8]:
end_time = datetime.datetime.now()

# Calculando la diferencia de tiempo
duration = end_time - start_time

print(f"Tiempo de ejecución: {duration}")

Tiempo de ejecución: 0:32:06.560338


In [9]:
len(train_histograms)

40000

In [10]:
len(train_labels)

40000

In [21]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.svm import SVC

def train_svm(histograms, labels):
    classifier = SVC(kernel='linear', probability=True)
    classifier.fit(histograms, labels)
    return classifier

from sklearn.naive_bayes import BernoulliNB

def train_naive_bayes(histograms, labels):
    # Uso de BernoulliNB que podría ser más adecuado para características binarias/histogramas
    classifier = make_pipeline(StandardScaler(), BernoulliNB())
    classifier.fit(histograms, labels)
    return classifier


from tensorflow.keras.layers import Dropout

def train_neural_network(histograms, labels):
    model = Sequential([
        Dense(512, activation='relu', input_shape=(histograms.shape[1],)),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(histograms, labels, epochs=40, batch_size=32, verbose=1)
    return model



In [12]:
start_time = datetime.datetime.now()
svm_classifier = train_svm(train_histograms, train_labels)
end_time = datetime.datetime.now()

# Calculando la diferencia de tiempo
duration = end_time - start_time

print(f"Tiempo de ejecución: {duration}")

Tiempo de ejecución: 1:32:21.835547


In [22]:
start_time = datetime.datetime.now()

nn_classifier = train_neural_network(np.array(train_histograms), np.array(train_labels))
    
end_time = datetime.datetime.now()

# Calculando la diferencia de tiempo
duration = end_time - start_time

print(f"Tiempo de ejecución: {duration}")

Epoch 1/40
1250/1250 [==============================] - 15s 9ms/step - loss: 0.6338 - accuracy: 0.6477
Epoch 2/40
1250/1250 [==============================] - 12s 9ms/step - loss: 0.6108 - accuracy: 0.6730
Epoch 3/40
1250/1250 [==============================] - 12s 9ms/step - loss: 0.5992 - accuracy: 0.6823
Epoch 4/40
1250/1250 [==============================] - 11s 9ms/step - loss: 0.5847 - accuracy: 0.6928
Epoch 5/40
1250/1250 [==============================] - 12s 9ms/step - loss: 0.5694 - accuracy: 0.7067
Epoch 6/40
1250/1250 [==============================] - 11s 9ms/step - loss: 0.5499 - accuracy: 0.7214
Epoch 7/40
1250/1250 [==============================] - 11s 9ms/step - loss: 0.5288 - accuracy: 0.7354
Epoch 8/40
1250/1250 [==============================] - 11s 9ms/step - loss: 0.5015 - accuracy: 0.7545
Epoch 9/40
1250/1250 [==============================] - 11s 9ms/step - loss: 0.4802 - accuracy: 0.7678
Epoch 10/40
1250/1250 [==============================] - 11s 9ms/step - l

In [23]:
start_time = datetime.datetime.now()

nb_classifier = train_naive_bayes(train_histograms, train_labels)
end_time = datetime.datetime.now()

# Calculando la diferencia de tiempo
duration = end_time - start_time

print(f"Tiempo de ejecución: {duration}")

Tiempo de ejecución: 0:00:01.207062


In [24]:
from sklearn.metrics import classification_report

def evaluate_classifiers(classifiers, test_histograms, test_labels):
    results = {}
    for name, classifier in classifiers.items():
        if name == 'Neural Network':
            predictions = classifier.predict(test_histograms) 
        else:
            predictions = classifier.predict(test_histograms)
        results[name] = classification_report(test_labels, predictions, target_names=['Class 0', 'Class 1'])
    return results


In [25]:
def load_and_extract_featuresT(folder, max_images_per_class=None, sift=cv2.SIFT_create()):
    images = []
    labels = []
    descriptors_list = []
    count_per_class = {}

    for label in sorted(os.listdir(folder)):
        class_folder = os.path.join(folder, label)
        count_per_class[label] = 0  # Inicializar contador para cada clase

        for filename in os.listdir(class_folder):
            if max_images_per_class is not None and count_per_class[label] >= max_images_per_class:
                continue  # Pasar a la siguiente clase si ya se alcanzó el límite

            img_path = os.path.join(class_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is not None:
                keypoints, descriptors = sift.detectAndCompute(img, None)
                if descriptors is not None:
                    images.append(img)
                    labels.append(int(label))
                    descriptors_list.append(descriptors)
                    count_per_class[label] += 1

            if max_images_per_class is not None and all(count >= max_images_per_class for count in count_per_class.values()):
                break  # Terminar completamente si todas las clases alcanzaron el límite

    return images, labels, descriptors_list


In [20]:
test_images, test_labels, test_descriptors = load_and_extract_featuresT(main_folder + "\\test", 500)
test_histograms = build_histograms(kmeans, test_descriptors)

# Evaluar los clasificadores
classifiers = {
    'SVM': svm_classifier,
    'Naive Bayes': nb_classifier,
    'Neural Network': nn_classifier
}
evaluation_results = evaluate_classifiers(classifiers, np.array(test_histograms), np.array(test_labels))

for result in evaluation_results:
    print(f"Results for {result}:")
    print(evaluation_results[result])


32/32 [==============================] - 0s 2ms/step
Results for SVM:
              precision    recall  f1-score   support

     Class 0       0.76      0.58      0.66       500
     Class 1       0.66      0.81      0.73       500

    accuracy                           0.70      1000
   macro avg       0.71      0.70      0.69      1000
weighted avg       0.71      0.70      0.69      1000

Results for Naive Bayes:
              precision    recall  f1-score   support

     Class 0       0.75      0.66      0.70       500
     Class 1       0.70      0.78      0.74       500

    accuracy                           0.72      1000
   macro avg       0.72      0.72      0.72      1000
weighted avg       0.72      0.72      0.72      1000

Results for Neural Network:
              precision    recall  f1-score   support

     Class 0       0.67      0.73      0.70       500
     Class 1       0.70      0.63      0.67       500

    accuracy                           0.68      1000
   ma

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    acc_history = {"train": [], "val": []}
    losses = {"train": [], "val": []}

    # we will keep a copy of the best weights so far according to validation accuracy
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    losses[phase].append(loss.item())

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).cpu().numpy()

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            
            acc_history[phase].append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, acc_history, losses



In [ ]:
def initialize_model(num_classes):
    # Resnet18 
    model = models.resnet18()
    
    model.fc = nn.Linear(512, num_classes)
    
    input_size = 224
        
    return model, input_size

In [ ]:

# Number of classes in the dataset
num_classes = 2

# Initialize the model
model, input_size = initialize_model(num_classes)

# Print the model we just instantiated
print(model)

# Send the model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [ ]:

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Number of epochs to train for 
num_epochs = 40

optimizer_ft = optim.Adam(model.parameters(), lr=0.001)



In [ ]:
# Train and evaluate
model, hist, losses = train_model(model, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs)

In [ ]:
# Plot the losses and accuracies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.plot(losses["train"], label="training loss")
ax1.plot(losses["val"], label="validation loss")
ax1.legend()

ax2.plot(hist["train"],label="training accuracy")
ax2.plot(hist["val"],label="val accuracy")
ax2.legend()

plt.show()

In [ ]:
subset_indices_test = torch.randperm(len(image_datasets['test']))[:int(0.3*len(image_datasets['test']))]
test_data_subset = torch.utils.data.Subset(image_datasets['test'], subset_indices_test)
test_dataloader = torch.utils.data.DataLoader(test_data_subset, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data).item()

    loss = running_loss / len(dataloader.dataset)
    accuracy = running_corrects / len(dataloader.dataset)

    return loss, accuracy

test_loss, test_accuracy = evaluate_model(model, test_dataloader)

print('Test Loss: {:.4f}, Test Accuracy: {:.4f}'.format(test_loss, test_accuracy))


In [ ]:
list_img_names = []

counter = 0
for i, (inputs, labels) in enumerate(test_dataloader):
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

    for j in range(inputs.size(0)):
        # Obtener el nombre de la imagen
        image_index = i * test_dataloader.batch_size + j
        image_path = test_data_subset.dataset.samples[image_index][0]
        image_name = os.path.basename(image_path)
        print("Nombre de la imagen: {}".format(image_name))
        list_img_names.append(image_name)
        # Loading and showing the image
        image = inputs[j].permute(1, 2, 0).cpu().numpy()

        # Normalizing the image
        image = (image - image.min()) / (image.max() - image.min())

        """
        plt.figure()
        plt.imshow(image)
        plt.axis('off')
        plt.show()
        """
        # Print the prediction and the correct label
        prediction = preds[j].item()
        correct_label = labels[j].item()
        print("Predicción: {}, Etiqueta correcta: {}".format(prediction, correct_label))

In [ ]:
patient_ids = []
for name in list_img_names:
    id = name.split('_')[0]
    patient_ids.append(id)

In [ ]:
ids_unique = list(set(patient_ids))

In [ ]:
base_path = main_folder

In [ ]:

def get_cancer_dataframe(patient_id, cancer_id):
    path = os.path.join(base_path, patient_id, cancer_id)
    files = os.listdir(path)
    dataframe = pd.DataFrame(files, columns=["filename"])
    path_names = [os.path.join(path, filename) for filename in dataframe.filename.values]
    dataframe = dataframe.filename.str.rsplit("_", n=4, expand=True)
    dataframe.loc[:, "target"] = np.int(cancer_id)
    dataframe.loc[:, "path"] = path_names
    dataframe = dataframe.drop([0, 1, 4], axis=1)
    dataframe = dataframe.rename({2: "x", 3: "y"}, axis=1)
    dataframe.loc[:, "x"] = dataframe.loc[:,"x"].str.replace("x", "", case=False).astype(np.int)
    dataframe.loc[:, "y"] = dataframe.loc[:,"y"].str.replace("y", "", case=False).astype(np.int)
    return dataframe

def get_patient_dataframe(patient_id):
    df_0 = get_cancer_dataframe(patient_id, "0")
    df_1 = get_cancer_dataframe(patient_id, "1")
    patient_df = pd.concat([df_0, df_1], ignore_index=True)
    return patient_df

In [ ]:
def visualise_breast_tissue_base(patient_id, pred_df=None):
    example_df = get_patient_dataframe(patient_id)
    max_point = [example_df.y.max()-1, example_df.x.max()-1]
    grid = 255*np.ones(shape=(max_point[0] + 50, max_point[1] + 50, 3)).astype(np.uint8)
    mask = 255*np.ones(shape=(max_point[0] + 50, max_point[1] + 50, 3)).astype(np.uint8)
    if pred_df is not None:
        patient_df = pred_df[pred_df.patient_id == patient_id].copy()
    mask_proba = np.zeros(shape=(max_point[0] + 50, max_point[1] + 50, 1)).astype(np.float)
    
    broken_patches = []
    for n in range(len(example_df)):
        try:
            image = imread(example_df.path.values[n])
            target = example_df.target.values[n]
            x_coord = np.int(example_df.x.values[n])
            y_coord = np.int(example_df.y.values[n])
            x_start = x_coord - 1
            y_start = y_coord - 1
            x_end = x_start + 50
            y_end = y_start + 50

            grid[y_start:y_end, x_start:x_end] = image
            if target == 1:
                mask[y_start:y_end, x_start:x_end, 0] = 250
                mask[y_start:y_end, x_start:x_end, 1] = 0
                mask[y_start:y_end, x_start:x_end, 2] = 0
            if pred_df is not None:
                proba = patient_df[(patient_df.x==x_coord) & (patient_df.y==y_coord)].proba
                mask_proba[y_start:y_end, x_start:x_end, 0] = np.float(proba)

        except ValueError:
            broken_patches.append(example_df.path.values[n])
    
    return grid, mask, broken_patches, mask_proba

In [ ]:
def visualise_breast_tissue(patient_id):
    grid, mask, broken_patches,_ = visualise_breast_tissue_base(patient_id)

    fig, ax = plt.subplots(1,2,figsize=(20,10))
    ax[0].imshow(grid, alpha=0.9)
    ax[1].imshow(mask, alpha=0.8)
    ax[1].imshow(grid, alpha=0.7)
    ax[0].grid(False)
    ax[1].grid(False)
    for m in range(2):
        ax[m].set_xlabel("X-coord")
        ax[m].set_ylabel("Y-coord")
    ax[0].set_title("Breast tissue slice of patient: " + patient_id)
    ax[1].set_title("Cancer tissue colored red \n of patient: " + patient_id)

    plt.show()

In [ ]:
def visualise_breast_tissue_binary(patient_id):
        
    fig, ax = plt.subplots(1, 1)

    example_df = get_patient_dataframe(patient_id)

    ax.scatter(example_df.x.values, example_df.y.values, c=example_df.target.values, cmap="coolwarm", s=20)
    ax.set_title("Patient " + patient_id)
    ax.set_xlabel("X coord")
    ax.set_ylabel("Y coord")
    ax.set_aspect('equal')  # Set aspect ratio to 'equal' to preserve original orientation
    ax.invert_yaxis()  # Reverse the y-axis direction

    plt.show()



In [ ]:
# Obtener 5 elementos aleatorios de la lista
random_patient_ids = random.sample(patient_ids, k=5)

In [ ]:
for id in random_patient_ids: #ids_unique:
    print("Patient's ID: ", id)
    visualise_breast_tissue(id)
    visualise_breast_tissue_binary(id)
    print("\n")